## 1 – Extraction des frames depuis une vidéo

In [ ]:
# Extraction de frames avec OpenCV
import cv2
import os

video_path = "match_sf6.mp4"
output_dir = "frames"
os.makedirs(output_dir, exist_ok=True)

cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_interval = int(fps // 10)  # ~10 images/sec

count, saved = 0, 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    if count % frame_interval == 0:
        frame_name = os.path.join(output_dir, f"frame_{saved:04d}.png")
        cv2.imwrite(frame_name, frame)
        saved += 1
    count += 1

cap.release()
print(f"✅ {saved} frames sauvegardées dans {output_dir}")


## 2 – Découpe du timer

In [ ]:
import matplotlib.pyplot as plt

# Coordonnées fixes pour un écran 1080p (à ajuster si nécessaire)
digit_boxes = [
    (860, 40, 40, 80),  # Chiffre 1
    (905, 40, 40, 80),  # Chiffre 2
]

frames_list = sorted(os.listdir(output_dir))
sample_frame = cv2.imread(os.path.join(output_dir, frames_list[0]))

plt.imshow(cv2.cvtColor(sample_frame, cv2.COLOR_BGR2RGB))
plt.title("Frame complète - Timer visible")
plt.show()

# Test de découpe
for (x, y, w, h) in digit_boxes:
    crop = sample_frame[y:y+h, x:x+w]
    plt.imshow(cv2.cvtColor(crop, cv2.COLOR_BGR2RGB))
    plt.show()


## 3 – Étiquetage assisté du dataset

In [4]:
from PIL import Image
import shutil
import os

dataset_dir = "dataset/train"
os.makedirs(dataset_dir, exist_ok=True)

for d in range(10):
    os.makedirs(os.path.join(dataset_dir, str(d)), exist_ok=True)

for frame_name in frames_list:
    frame_path = os.path.join(output_dir, frame_name)
    frame = cv2.imread(frame_path)
    for i, (x, y, w, h) in enumerate(digit_boxes):
        crop = frame[y:y+h, x:x+w]
        plt.imshow(cv2.cvtColor(crop, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.show()
        label = input(f"Chiffre #{i+1} : ")
        if label.isdigit() and 0 <= int(label) <= 9:
            save_path = os.path.join(dataset_dir, label, f"{frame_name}_digit{i}.png")
            cv2.imwrite(save_path, crop)


NameError: name 'frames_list' is not defined

## 4 – Entraînement du modèle

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.ImageFolder(dataset_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

model = models.resnet18(pretrained=True)
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
model.fc = nn.Linear(model.fc.in_features, 10)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 5
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader):.4f}")

torch.save(model.state_dict(), "sf6_timer_model.pth")
print("✅ Modèle sauvegardé")


## 5 – Prédiction sur une image de match

In [ ]:
def predict_digit(img_crop):
    img = transform(Image.fromarray(cv2.cvtColor(img_crop, cv2.COLOR_BGR2RGB))).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(img)
        _, pred = torch.max(output, 1)
    return str(pred.item())

test_image = cv2.imread("test_frame.png")
recognized = []
for (x, y, w, h) in digit_boxes:
    crop = test_image[y:y+h, x:x+w]
    recognized.append(predict_digit(crop))

print("⏱ Timer reconnu :", "".join(recognized))
